In [2]:
from utils.thai_lottery_scraper import ThaiLotteryScraper

df = ThaiLotteryScraper().scrape_df(35)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               840 non-null    datetime64[ns]
 1   วัน                840 non-null    object        
 2   เดือน              840 non-null    object        
 3   ปี                 840 non-null    object        
 4   รางวัลที่1         840 non-null    object        
 5   2ตัวบน             840 non-null    object        
 6   3ตัวบน             840 non-null    object        
 7   2ตัวล่าง           840 non-null    object        
 8   3ตัวหน้า,3ตัวล่าง  840 non-null    object        
 9   3ตัวหน้า           840 non-null    object        
 10  3ตัวล่าง           840 non-null    object        
dtypes: datetime64[ns](1), object(10)
memory usage: 72.3+ KB


In [34]:
import pandas as pd

df_1prize = pd.DataFrame(df['รางวัลที่1'].apply(lambda x: [{"digit":i+1,"value":digit} for i, digit in enumerate(x)]).explode().reset_index(drop=True).to_list())
df_1prize.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5156 entries, 0 to 5155
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   digit   5156 non-null   int64 
 1   value   5156 non-null   object
dtypes: int64(1), object(1)
memory usage: 80.7+ KB


In [35]:
df_1prize.groupby('digit')['value'].value_counts().sort_values(ascending=False).reset_index().to_csv('1prize_digit_value_counts.csv', index=False, header=False)

In [29]:

df_1prize = pd.DataFrame(df['รางวัลที่1'].apply(lambda x: [
                         {"value [i]": x[i], "value [i+1]": x[i+1]} for i in range(len(x)-1)]).explode().reset_index(drop=True).to_list())
df_1prize.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4316 entries, 0 to 4315
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   value [i]    4316 non-null   object
 1   value [i+1]  4316 non-null   object
dtypes: object(2)
memory usage: 67.6+ KB


In [33]:
df_1prize.groupby('value [i]')['value [i+1]'].value_counts().sort_values(ascending=False).reset_index(
).to_csv('1prize_what_next.csv', index=False, header=False)